In [13]:
!pip install nltk

In [14]:
import nltk

In [15]:
import pandas as pd
from nltk.tokenize import TweetTokenizer
import numpy as np

In [16]:
from pymongo import MongoClient
import re

In [17]:
client= MongoClient('mongo', 27017, username='admin', password='DataMan2019!')

In [20]:
db = list(client.Progetto_dm.Database_pulito.find())
tweets = pd.DataFrame(db)
tweets.head()

,_id,text,hashtags
0,609909ba97f00aae66f41d39,"RT @gaysforkim: TW // bombing, shooting, attac...",[]
1,609909ba97f00aae66f41d3a,Our regularly updated interactive databank hel...,[]
2,609909ba97f00aae66f41d3b,RT @ozget0rer: Forgiveness is the key to letti...,[]
3,609909ba97f00aae66f41d3c,RT @NCTsmtown_DREAM: It’s finally today!! We p...,[]
4,609909ba97f00aae66f41d3d,someone brought this back on my tl and i for r...,[]


In [21]:
nltk.download(["stopwords","vader_lexicon"])

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

La seguente funzione rimuove: le emoticons, la punteggiatura, simboli come # e RT, i collegamenti ipertestuali ed un corpus di parole che NLTK chiama “Stopwords” cioè quelle parole che fungono da congiunzione ma non aggiungono significato alla frase.

In [22]:
import string
import re

from nltk.corpus import stopwords 
stopwords_english = stopwords.words('english')

from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

from nltk.tokenize import TweetTokenizer

# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

# all emoticons (happy + sad)
emoticons = emoticons_happy.union(emoticons_sad)

def clean_tweets(tweet):
	# remove stock market tickers like $GE
	tweet = re.sub(r'\$\w*', '', tweet)

	# remove old style retweet text "RT"
	tweet = re.sub(r'^RT[\s]+', '', tweet)

	# remove hyperlinks
	tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
	
	# remove hashtags
	# only removing the hash # sign from the word
	tweet = re.sub(r'#', '', tweet)

	# tokenize tweets
	tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
	tweet_tokens = tokenizer.tokenize(tweet)

	tweets_clean = []	
	for word in tweet_tokens:
		if (word not in stopwords_english and # remove stopwords
			  word not in emoticons and # remove emoticons
			    word not in string.punctuation): # remove punctuation
			#tweets_clean.append(word)
			stem_word = stemmer.stem(word) # stemming word
			tweets_clean.append(stem_word)

	return tweets_clean

In [23]:
tweets['puliti'] = tweets['text'].apply(lambda x: clean_tweets(x))

Adesso effettuo la sentiment delle frasi, come media della sentiment delle singole parole che le compongono

In [24]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
tweets['puliti2'] = tweets['puliti'].str.join(" ")
tweets['score1'] = tweets['puliti2'].apply(lambda x: sia.polarity_scores(x))

In [25]:
import numpy as np
tweets['score2'] = tweets['score1']
tweets.head()

,_id,text,hashtags,puliti,puliti2,score1,score2
0,609909ba97f00aae66f41d39,"RT @gaysforkim: TW // bombing, shooting, attac...",[],"[tw, bomb, shoot, attack, know, what', happen,...",tw bomb shoot attack know what' happen rn pl l...,"{'neg': 0.442, 'neu': 0.558, 'pos': 0.0, 'comp...","{'neg': 0.442, 'neu': 0.558, 'pos': 0.0, 'comp..."
1,609909ba97f00aae66f41d3a,Our regularly updated interactive databank hel...,[],"[regularli, updat, interact, databank, help, s...",regularli updat interact databank help see pla...,"{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'comp...","{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'comp..."
2,609909ba97f00aae66f41d3b,RT @ozget0rer: Forgiveness is the key to letti...,[],"[forgiv, key, let, go, forgiv, releas, releas]",forgiv key let go forgiv releas releas,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
3,609909ba97f00aae66f41d3c,RT @NCTsmtown_DREAM: It’s finally today!! We p...,[],"[’, final, today, prepar, lot, comeback, despe...",’ final today prepar lot comeback desper show ...,"{'neg': 0.0, 'neu': 0.783, 'pos': 0.217, 'comp...","{'neg': 0.0, 'neu': 0.783, 'pos': 0.217, 'comp..."
4,609909ba97f00aae66f41d3d,someone brought this back on my tl and i for r...,[],"[someon, brought, back, tl, real, broke, tear,...",someon brought back tl real broke tear jisung ...,"{'neg': 0.237, 'neu': 0.763, 'pos': 0.0, 'comp...","{'neg': 0.237, 'neu': 0.763, 'pos': 0.0, 'comp..."


Dopo esserci documentati al riguardo, abbiamo scelto di non affidarci al valore della voce compound ma di creare una nuova colonna di score che contenesse il valore più alto tra positive e negative o il valore neutral quando nessuno dei due prevaleva sull’altro.

In [26]:
from progressbar import ProgressBar
pbar = ProgressBar()

for i in pbar(range(len(tweets['score1']))):
    tweets['score2'][i] = np.where((tweets['score1'][i]['pos'] > tweets['score1'][i]['neg']), tweets['score1'][i]['pos'], 
                                 (np.where((tweets['score1'][i]['neg'] > tweets['score1'][i]['pos']), f"-{tweets['score1'][i]['neg']}", 0)))


KeyboardInterrupt: 

Rimuoviamo le colonne che ci sono servite solo per le lavorazioni.

In [20]:
tweets.drop(["puliti", "puliti2"], axis=1, inplace= True)
tweets.head()

Trasformiano la colonna score2 in float. Prima era un array Numpy.

In [ ]:
tweets['score2'] = tweets['score2'].astype(float)

In [21]:
tweets_d = tweets.to_dict(orient='records')
tweets_d.head()

In [22]:
client.Progetto_dm.Database_pulito_sentiment.insert_many(tweets_d)